<a href="https://colab.research.google.com/github/lgeralas/Actividad-4/blob/main/Actividad_4_Regresi%C3%B3n_log%C3%ADstica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
#Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
 #cargar archivo csv desde equipo 
from google.colab import files 
files.upload()

In [24]:
#Carga desde un archivo .csv sin indice
microretailer = pd.read_excel('microretailer_mit_lift_lab_actualizado.xlsx')

In [ ]:
#A través de un filtro se buscaran todas aquellas entrevistas que su tienda es de tipo tienda de conveniencia
filtro1=microretailer[microretailer["232_type_of_store"] == "Micro-restaurant (aka. fondita)"]
filtro1

In [54]:
#Para fines más rápidos se propone el llenar todos los especios nulos con la opción de llenado hacia adelante 
#Este filtrado nos ayudará a realizar las 3 regresiones lineales solicutadas, utilizando unicamente los valores númericos
filtros =filtro1.fillna(method="ffill")
filtros['186_internet_connection']=filtros['186_internet_connection'].fillna('yes')
filtros['310_burnout']=filtros['310_burnout'].fillna('yes')
filtros

,_record_id,_title,_server_updated_at,_updated_by,_geometry,_latitude,_longitude,228_store_name,229_store_picture,232_type_of_store,...,261_technology_organization,262_technology_serve_customers,263_technology_secure_sales,264_technology_budget,267_technology_investment_customer_preferences,234_willingness_participate_workshop,318_shopkeeper_name,319_shopkeepers_last_name,236_whatsapp,235_email
4,3c2177cc-677b-4106-8ed2-6e50f7671a0e,"November 17, 2022",2022-11-18 13:30:19 UTC,Fernando Báez,SRID=4326;POINT(-98.201134218 19.0436743967),19.043674,-98.201134,Antojitos Tony,1 Photo,Micro-restaurant (aka. fondita),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,6f361b49-fc33-499a-9ca0-12e1cf79ea81,"November 17, 2022",2022-11-18 13:30:18 UTC,Fernando Báez,SRID=4326;POINT(-98.2013931652 19.043971536),19.043972,-98.201393,Tortas 7 hermanos,1 Photo,Micro-restaurant (aka. fondita),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8ed941a0-46da-48bc-9205-67125f3ec5a1,"November 17, 2022",2022-11-18 13:30:17 UTC,Fernando Báez,SRID=4326;POINT(-98.2017912559 19.0439668372),19.043967,-98.201791,La margarita,1 Photo,Micro-restaurant (aka. fondita),...,Totalmente de acuerdo,De acuerdo,Totalmente de acuerdo,Ni de acuerdo ni en desacuerdo,De acuerdo,no,NaN,NaN,NaN,NaN
12,fb9301dd-ec43-4bc4-8f50-0e1a9dbe84ad,"November 8, 2022",2022-11-18 04:35:14 UTC,Cristian Contreras,SRID=4326;POINT(-98.2212818134 19.0417481554),19.041748,-98.221282,Cemitas Paty,1 Photo,Micro-restaurant (aka. fondita),...,Ni de acuerdo ni en desacuerdo,En desacuerdo,Totalmente en desacuerdo,En desacuerdo,Ni de acuerdo ni en desacuerdo,no,NaN,NaN,NaN,NaN
19,fa7bfebc-72bb-4fe7-8ef3-26744805dab3,"November 8, 2022",2022-11-18 04:35:10 UTC,Cristian Contreras,SRID=4326;POINT(-98.2214231845 19.0414639052),19.041464,-98.221423,Sabores de mi tierra,1 Photo,Micro-restaurant (aka. fondita),...,Totalmente en desacuerdo,Ni de acuerdo ni en desacuerdo,Totalmente en desacuerdo,De acuerdo,De acuerdo,no,NaN,NaN,NaN,NaN
21,0bfafebf-8c79-432b-904e-a24584b87f2a,"November 8, 2022",2022-11-18 04:35:08 UTC,Cristian Contreras,SRID=4326;POINT(-98.2212611005 19.0418224803),19.041822,-98.221261,El buen sazón,1 Photo,Micro-restaurant (aka. fondita),...,Ni de acuerdo ni en desacuerdo,Ni de acuerdo ni en desacuerdo,En desacuerdo,Totalmente en desacuerdo,De acuerdo,no,NaN,NaN,NaN,NaN
25,86b1ce5f-bb76-4ab3-86fb-01b243452cd5,"November 8, 2022",2022-11-18 04:35:06 UTC,Cristian Contreras,SRID=4326;POINT(-98.2218344306 19.0408630157),19.040863,-98.221834,Cazuelas el apapacho,1 Photo,Micro-restaurant (aka. fondita),...,De acuerdo,De acuerdo,Ni de acuerdo ni en desacuerdo,Totalmente en desacuerdo,En desacuerdo,yes,NaN,NaN,2.221904e+09,NaN
26,8ae5eaec-c3c3-4022-8c6d-7a3b73f9ebd9,"November 4, 2022",2022-11-18 04:35:05 UTC,Cristian Contreras,SRID=4326;POINT(-98.2164242172 19.0472081696),19.047208,-98.216424,Las tortas del barrio,1 Photo,Micro-restaurant (aka. fondita),...,De acuerdo,Totalmente de acuerdo,De acuerdo,De acuerdo,De acuerdo,yes,Jessica,Villalobos,2.223245e+09,NaN
28,840c4938-6e15-4004-b46f-0d179286663c,"November 8, 2022",2022-11-18 04:35:04 UTC,Cristian Contreras,SRID=4326;POINT(-98.2213921543 19.0416289221),19.041629,-98.221392,HooDoo Brown,1 Photo,Micro-restaurant (aka. fondita),...,De acuerdo,Ni de acuerdo ni en desacuerdo,Ni de acuerdo ni en desacuerdo,En desacuerdo,En desacuerdo,no,Jessica,Villalobos,2.223245e+09,NaN
31,247b33fe-9dd4-4e91-8815-463e0e7bdd84,"November 8, 2022",2022-11-18 04:35:03 UTC,Cristian Contreras,SRID=4326;POINT(-98.2203426225 19.0434835111),19.043484,-98.220343,La fogata,1 Photo,Micro-restaurant (aka. fondita),...,Ni de acuerdo ni en desacuerdo,Ni de acuerdo ni en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,De acuerdo,no,Jessica,Villalobos,2.223245e+09,NaN


**Eliminación de outliers para valores numéricos**

In [ ]:
#Filtramos todos los datos númericos una vez que ya tenemos a todos los datos donde el tiepo de tienda es 'grocery'
filtro2=filtro1.iloc[:,[13,14,16,21,22,29,31,33,41,42,43,44,45,49,51,56,57,58,59,60,78,90]] 
filtro2

In [56]:
#Método aplicando desviación estandar. Encuentra los valores extremos 
y=filtro2
Limite_Superior= y.mean() + 3*y.std()
Limite_Inferior= y.mean() - 3*y.std()
print('Limite superior permitido', Limite_Superior)
print('Limite inferior permitido', Limite_Inferior)

Limite superior permitido 97_number_of_customers_in_store                       9.089698
268_number_fridges                                    4.279779
104_how_many_shelves_does_the_micro_retailer_have     7.914450
2_current_permanent_employees                        11.145449
4_number_permanent_employees_last_year               13.210855
103_number_own_fridges                                4.534963
145_number_direct_competitors                        20.436282
24_burnout                                           10.187278
272_card_days_receive_money                           3.355587
275_platform_days_receive_money                      13.816654
276_expected_days_informal_credit                    46.653151
36_credit_default_rate                                8.671372
315_frequency_topups                                       NaN
316_frequency_utility_bills                                NaN
317_home_deliveries                                  44.759302
62_frequency_only_services   

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  after removing the cwd from sys.path.


In [57]:
#Obtenemos datos limpiosy los outliers se convertiran en nulos en el Dataframe
filtro2=filtro2[(y<=Limite_Superior)&(y>=Limite_Inferior)]

#Reemplazar valores átipicos (nulos) del dataframe con la opción de llenado hacia adelante
filtro2=filtro2.fillna(round(filtros.mean(),1))
filtro2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version.  Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """


,97_number_of_customers_in_store,268_number_fridges,104_how_many_shelves_does_the_micro_retailer_have,2_current_permanent_employees,4_number_permanent_employees_last_year,103_number_own_fridges,145_number_direct_competitors,24_burnout,271_card_fee,272_card_days_receive_money,...,275_platform_days_receive_money,276_expected_days_informal_credit,36_credit_default_rate,315_frequency_topups,316_frequency_utility_bills,317_home_deliveries,62_frequency_only_services,78_number_home_deliveries_week,172_supplier_frequency,163_number_high_perishable_products
4,3.0,2.0,3.0,2.7,2.3,1.5,4.3,3.0,NaN,1.1,...,4.5,11.7,1.4,NaN,NaN,16.5,15.2,NaN,3.0,8.4
7,3.0,2.0,2.0,2.7,2.3,1.5,4.3,3.0,NaN,1.1,...,4.5,11.7,1.4,NaN,NaN,16.5,15.2,NaN,3.0,8.4
8,6.0,4.0,3.0,10.0,2.3,4.0,3.0,3.0,NaN,0.0,...,0.0,11.7,1.4,NaN,NaN,4.0,0.0,NaN,3.0,30.0
12,1.0,1.0,1.0,2.0,3.0,0.0,4.0,3.0,NaN,1.1,...,4.5,11.7,1.4,NaN,NaN,16.5,15.2,NaN,2.0,10.0
19,2.0,1.0,0.0,3.0,2.0,1.0,3.0,1.0,NaN,1.1,...,4.5,11.7,1.4,NaN,NaN,16.5,15.2,NaN,2.0,3.0
21,1.0,2.0,1.0,2.0,2.0,1.0,5.0,3.0,NaN,1.1,...,4.5,11.7,1.4,NaN,NaN,16.5,15.2,NaN,1.0,5.0
25,3.0,2.0,2.0,2.0,2.0,2.0,8.0,3.0,NaN,1.1,...,4.5,11.7,1.4,NaN,NaN,16.5,15.2,NaN,1.0,3.0
26,3.0,1.0,1.0,2.0,3.0,1.0,10.0,1.0,NaN,1.0,...,4.5,11.7,1.4,NaN,NaN,10.0,10.0,NaN,2.0,6.0
28,1.0,1.0,1.0,1.0,2.0,1.0,3.0,1.0,NaN,1.1,...,4.5,11.7,1.4,NaN,NaN,12.0,12.0,NaN,2.0,3.0
31,2.0,2.0,2.0,4.0,3.0,2.0,3.0,3.0,NaN,1.0,...,4.5,11.7,1.4,NaN,NaN,16.5,15.2,NaN,1.0,10.0


In [58]:
#Unimos los datos númericos con los datos cualitativos en una misma tabla
filtros['97_number_of_customers_in_store']=filtro2['97_number_of_customers_in_store']
filtros['268_number_fridges']=filtro2['268_number_fridges']
filtros['104_how_many_shelves_does_the_micro_retailer_have']=filtro2['104_how_many_shelves_does_the_micro_retailer_have']
filtros['2_current_permanent_employees']=filtro2['2_current_permanent_employees']
filtros['4_number_permanent_employees_last_year']=filtro2['4_number_permanent_employees_last_year']
filtros['103_number_own_fridges']=filtro2['103_number_own_fridges']
filtros['145_number_direct_competitors']=filtro2['145_number_direct_competitors']
filtros['24_burnout']=filtro2['24_burnout']
filtros['276_expected_days_informal_credit']=filtro2['276_expected_days_informal_credit']
filtros['272_card_days_receive_money']=filtro2['272_card_days_receive_money']
filtros['36_credit_default_rate']=filtro2['36_credit_default_rate']
filtros['315_frequency_topups']=filtro2['315_frequency_topups']
filtros['316_frequency_utility_bills']=filtro2['316_frequency_utility_bills']
filtros['317_home_deliveries']=filtro2['317_home_deliveries']
filtros['62_frequency_only_services']=filtro2['62_frequency_only_services']
filtros['78_number_home_deliveries_week']=filtro2['78_number_home_deliveries_week']
filtros['172_supplier_frequency']=filtro2['172_supplier_frequency']
filtros['163_number_high_perishable_products']=filtro2['163_number_high_perishable_products']

**Regresión logística**

Se procederá a realizar una regresión logística donde se analizarán diferentes casos de predicción y así saber que tan preciso, exacto y sensible es el modelo creado.

- En el primer análisis se creaará un modelo que prediga si la tienda vende productos frescos (Si/no), esto con ayuda de las variables independientes (# de estantes y # de refrigeradores)

In [71]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= filtros[['268_number_fridges', '104_how_many_shelves_does_the_micro_retailer_have']]
Var_Dep= filtros['102_does_the_micro_retailer_sells_fresh_products_']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 
y_pred

array(['no', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes',
       'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes'],
      dtype=object)

In [72]:
#Cálculo de presición, exactitud y sensibilidad

#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="yes")
print('Precisión del modelo:')
print(precision)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="yes")
print('Sensibilidad del modelo:')
print(sensibilidad)

Precisión del modelo:
0.9375
Exactitud del modelo:
0.8888888888888888
Sensibilidad del modelo:
0.9375


- El siguinete modelo predecirá si el lugar cuenta con conexión a internet, haciendo uso de las variables independientes (clientes en la tienda y # de empleados)

In [67]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= filtros[['97_number_of_customers_in_store', '4_number_permanent_employees_last_year']]
Var_Dep= filtros['186_internet_connection']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 
y_pred

array(['yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'no',
       'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes'],
      dtype=object)

In [68]:
#Cálculo de presición, exactitud y sensibilidad

#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="yes")
print('Precisión del modelo:')
print(precision)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="yes")
print('Sensibilidad del modelo:')
print(sensibilidad)

Precisión del modelo:
0.7058823529411765
Exactitud del modelo:
0.6666666666666666
Sensibilidad del modelo:
0.9230769230769231


- Para este último se realizará un modelo que pueda predecir que la persona que atiende dentro de la tienda siente algún tipo de cansancio, esto dependiento de el número de clientes la cantidad de productos perecederos que tiene la misma tienda

In [63]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= filtros[['97_number_of_customers_in_store', '163_number_high_perishable_products']]
Var_Dep= filtros['310_burnout']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 
y_pred

array(['no', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'yes', 'no',
       'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes'],
      dtype=object)

In [64]:
#Cálculo de presición, exactitud y sensibilidad

#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="yes")
print('Precisión del modelo:')
print(precision)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="yes")
print('Sensibilidad del modelo:')
print(sensibilidad)

Precisión del modelo:
0.42857142857142855
Exactitud del modelo:
0.4444444444444444
Sensibilidad del modelo:
0.75
